In [3]:
from bs4 import BeautifulSoup

In [4]:
import requests

In [5]:
import pandas as pd
import re

In [6]:
df = pd.read_csv('links.csv', delimiter= '\t')

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [8]:
# initialize abstracts list and citations list
abstracts = []
all_references = [] # nested list

In [9]:
# make sure reference link exists 
def check_ref(ref):
    try:
        link = ref.ul.li.a.get('href')
        if link != '#': # occurs if link is fake
            return link
    except AttributeError: # occurs if no link exists
        return

In [10]:
def main():
    for link in df.link:
                
        page = requests.get(link, headers=headers).content
        soup = BeautifulSoup(page, 'html.parser')
        
        # get abstract 
        abstract = soup.find('div', 'abstract-content').p.get_text()
        abstracts.append(abstract)
        
        # get references
        # initialize reference list
        local_references = []
        references = soup.find('ol', 'references').find_all('li', id=re.compile('^ref\d+'))
        for ref in references:
            checked_ref = check_ref(ref)
            if checked_ref != None:
                local_references.append(checked_ref)
        all_references.append(local_references)
        


In [11]:
main()

c:\users\danny\miniconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'journals.plos.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\danny\miniconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'journals.plos.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\danny\miniconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'journals.plos.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\danny\miniconda3\lib\site-packages\urlli

In [12]:
df["abstract"] = abstracts
df["references"] = all_references

In [13]:
df.to_csv(r'results.csv', sep='\t', index = False)

In [14]:
len(all_references)

13

In [15]:
count = 0
for listElem in all_references:
    count += len(listElem)
    
print(count)

331
